In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Union

In [ ]:
M = 1.989e30 #kg
G = 6.6741e-11 # m^3/kg/s^2
AU = 149597870700 #m
m = 1 #comet mass

In [ ]:
x0 = np.array([0, 0.586]) * AU # m
v0 = np.array([54600, 0]) # m/s


In [ ]:
def V(r: np.ndarray) -> np.ndarray:
    """
    Gravitonal Potential Energy dependent on position r = [x, y]
    """
    return -G * M * m * r  / (np.power(r.dot(r), 3/2))# J

In [ ]:
def derivative(func, x0: float, n: int = 1) -> float:
    """
    Computes derivative of a function f where x=x0, using recurrency
    """
    dx = 1e-6
    if n == 1:
        dy = func(x0+dx) - func(x0)
        return dy/dx
    if n > 1:
        dy = derivative(func, x0 + dx, n-1) - derivative(func, x0, n-1)
        return dy / dx

In [ ]:
class Solver:
    def __init__(self, tmax: int, dt: float, alpha: float, mass: float):
        self.x = None
        self.v = None
        self.t = None
        self.alpha = None
        self.mass = None
        self.dt = None
        self.update_params(tmax, dt, alpha, mass)

    def update_params(self, tmax: int, dt: float, alpha: float, mass: float = M):
        num = int(tmax//dt)
        self.x = np.zeros(num)
        self.v = np.zeros(num)
        self.t = np.linspace(0, tmax, num=num)
        self.alpha = alpha
        self.mass = mass
        self.dt = dt

    def set_new_values(self, x: float, v: float, i):
        self.x[i] = x
        self.v[i] = v

    def get_new_values(self, i: int):
        return self.x[i], self.y[i]

    def calculate(self):
        for i in range(len(self.t)):
            if i == 0:
                self.set_new_values(x0, v0, 0)
                continue
            x, y = self.get_next_step(i-1)
            self.set_new_values(x, y, i)

In [ ]:
class Euler(Solver):
    def __init__(self, tmax: int, dt: float, alpha: float, mass: float = M):
        super().__init__(tmax, dt, alpha, mass)

    def get_next_step(self, i: int):
        new_x = self.x[i] + self.v[i] * self.dt
        new_y = self.v[i] - 1 / self.mass * derivative(V, self.x[i]) * self.dt - self.alpha * self.v[i] * self.dt
        return new_x, new_y